In [1]:
import sys
import gc
import keras
import numpy as np
import numpy
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
sys.path.append('../')
from wiki.utils import clock

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, GRU
from keras.optimizers import Adam

In [4]:
base_dir = '../data/'

In [5]:
with clock():
    train = pd.read_csv(base_dir+'train_1.csv').fillna(0)

Elapsed time 12.679872751235962 seconds


In [6]:
len(train.columns)

551

In [7]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [8]:
with clock():
    scaler = MinMaxScaler(feature_range=(0, 1))
    sequences = [row.values[1:] for _,row in train.iterrows()]

Elapsed time 10.61281681060791 seconds


In [9]:
del train
gc.collect()

348

In [10]:
with clock():
    sequences = np.transpose(scaler.fit_transform(np.transpose(sequences)))

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


Elapsed time 7.169586658477783 seconds


In [11]:
time_steps = 60

In [12]:
train_len = len(sequences[0])-time_steps ; train_len

490

In [13]:
def get_data_sequence(sequence, train_len):
    sequences = []
    next_elements = []
    for i in range(len(sequence)-train_len+1):
        sequences.append(sequence[i:i+train_len])
        next_elements.append(sequence[i+1:i+1+train_len])
    return np.stack(sequences[:-1]), np.stack(next_elements[:-1])

In [14]:
def get_data(sequences, train_len):
    return_sequences = []
    next_elements = []
    for s in sequences:
        ss, nes = get_data_sequence(s, train_len)
        return_sequences.append(ss)
        next_elements.append(nes)
    return_sequences = np.expand_dims(np.concatenate(return_sequences), axis=-1)
    next_elements = np.expand_dims(np.concatenate(next_elements), axis=-1)
    return return_sequences, next_elements

In [15]:
def data_gen(sequences, train_len, batch_size):
    while True:
        return_sequences, next_elements = [], []
        for s in sequences:
            for i in range(len(s)-train_len):
                return_sequences.append(s[i:i+train_len])
                next_elements.append(s[i+1:i+1+train_len])
            if len(sequences) == batch_size:
                stack_expand = lambda x: np.expand_dims(np.stack(x), axis=-1)
                yield stack_expand(return_sequences), stack_expand(next_elements)
                return_sequences, next_elements = [], []

In [16]:
batch_size = 128
n_hidden = 256

In [18]:
model=Sequential([
        LSTM(n_hidden, batch_input_shape=(batch_size, train_len, 1), return_sequences=True, dropout_U=0.2, dropout_W=0.2,
            implementation=2),
        Dropout(0.2),
        TimeDistributed(Dense(1))
    ])    

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, batch_input_shape=(128, 490,..., return_sequences=True, implementation=2, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, return_sequences=True, implementation=2, dropout=0.2, recurrent_dropout=0.2)`
  


In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (128, 490, 256)           264192    
_________________________________________________________________
dropout_3 (Dropout)          (128, 490, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (128, 490, 256)           525312    
_________________________________________________________________
dropout_4 (Dropout)          (128, 490, 256)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (128, 490, 1)             257       
Total params: 789,761.0
Trainable params: 789,761.0
Non-trainable params: 0.0
_________________________________________________________________


In [20]:
#model.summary() #Old model summary (before stateful was True, so batch size didn't have to be prespecified)

In [ ]:
model.compile(loss='mean_squared_error', optimizer=Adam())

In [ ]:
model.fit_generator(
    data_gen(sequences, train_len, batch_size), 
    steps_per_epoch=len(sequences)*time_steps // batch_size,
    epochs=1
) 

Epoch 1/1


In [ ]:
#model.fit(train_sequences, next_elements, batch_size=64, epochs=1) #Old model training w/ stateful=False)

Epoch 1/1
3520/6000 [================>.............] - ETA: 452s - loss: 0.0101